In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install  python-levenshtein 
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from scipy import spatial
from tqdm import tqdm
import Levenshtein
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS.union(["book"])

     |████████████████████████████████| 51kB 3.4MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144672 sha256=7c3f9947123d42e8dcb7a3acd0ed723f4af1924a7399df2d8497e6cd31a4d0f0
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
df=pd.read_csv('/content/drive/My Drive/TF_QA_DATA_SQuAD_FORMAT/TF_QA_DATA_SQuAD_FORMAT.csv')
df.fillna('',inplace=True)

In [0]:
df.head(5)

,example_id,text,candidate_answers,question,pred_short_ans,pred_long_ans,pred_yes_no_ans,doc_url
0,5655493461695504401,Email marketing - Wikipedia <H1> Email marketi...,['<Table> <Tr> <Td> </Td> <Td> ( hide ) This a...,which is the most common use of opt-in e-mail ...,a newsletter sent to an advertising firm 's cu...,<P> A common example of permission marketing i...,,https://en.wikipedia.org//w/index.php?title=Em...
1,5328212470870865242,The Mother ( How I Met Your Mother ) - wikiped...,"['<Table> <Tr> <Th_colspan=""2""> Tracy McConnel...",how i.met your mother who is the mother,Tracy McConnell,"<P> Tracy McConnell , better known as `` The M...",,https://en.wikipedia.org//w/index.php?title=Th...
2,4435104480114867852,Human fertilization - wikipedia <H1> Human fer...,['<Table> <Tr> <Td> Part of a series on </Td> ...,what type of fertilisation takes place in humans,,<P> The process of fertilization involves a sp...,,https://en.wikipedia.org//w/index.php?title=Hu...
3,5289242154789678439,List of National Football League career quarte...,"[""<Table> <Tr> <Th> NFL lists </Th> </Tr> <Tr>...",who had the most wins in the nfl,Tom Brady,<P> Active quarterback Tom Brady holds the rec...,,https://en.wikipedia.org//w/index.php?title=Li...
4,5489863933082811018,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[""<Table> <Tr> <Td> </Td> <Td> This article 's...",what happened to the lost settlement of roanoke,,,,https://en.wikipedia.org//w/index.php?title=Ro...


In [0]:
def format_txt(x):
  if x!='':
    x=[' '.join(x1.split()) for x1 in x]
    return x
  else:
    return x

def format_text(x):
  if x!='':
    x=' '.join(x.split())
    return x
  else:
    return x

def remove_html_tag(text):
    """Remove html tags from a string"""
    import re
    if len(text)>=0:
      clean = re.compile('<.*?>')
      text=[re.sub(clean, '', text1) for text1 in text]
      return text
    else: 
      return text

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    if text!='':
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    else:
      return text

def jaccard_similarity(query,document):
  intersection=set(query).intersection(set(document))
  union=set(query).union(set(document))
  index=len(intersection)/len(union)
  return index

In [0]:
def extract_features(text_tfidf,ques_tfidf,ans_tfidf,text,ques,ans):

  qa_cos_d = 1-spatial.distance.cosine(ques_tfidf,ans_tfidf)
  qd_cos_d = 1-spatial.distance.cosine(ques_tfidf,text_tfidf)
  ad_cos_d = 1-spatial.distance.cosine(ans_tfidf,text_tfidf)

  qa_euc_d = spatial.distance.euclidean(ques_tfidf,ans_tfidf)
  qd_euc_d = spatial.distance.euclidean(ques_tfidf,text_tfidf)
  ad_euc_d = spatial.distance.euclidean(ans_tfidf,text_tfidf)

  qa_lev_d = Levenshtein.distance(ques, ans)
  qa_lev_r = Levenshtein.ratio(ques, ans)
  qa_jar_s = Levenshtein.jaro(ques, ans) 
  qa_jaw_s = Levenshtein.jaro_winkler(ques, ans)

  qa_tfidf_score=np.sum(ques_tfidf*ans_tfidf.T)
  qd_tfidf_score=np.sum(ques_tfidf*text_tfidf.T)
  ad_tfidf_score=np.sum(ans_tfidf*text_tfidf.T)

  ques_tfidf_sum=np.sum(ques_tfidf)
  txt_tfidf_sum=np.sum(text_tfidf)
  ans_tfidf_sum=np.sum(ans_tfidf)

  lemmatizer=WordNetLemmatizer()
  ans=[lemmatizer.lemmatize(x1,pos="a") for x1 in ans.split()]
  ques=[lemmatizer.lemmatize(x,pos="a") for x in ques.split()]
  text=[lemmatizer.lemmatize(x,pos="a") for x in text.split()]

  qa_jacc_simi=jaccard_similarity(ques,ans)
  qd_jacc_simi=jaccard_similarity(ques,text)
  ad_jacc_simi=jaccard_similarity(ans,text)

  return [qa_cos_d, qd_cos_d, ad_cos_d, qa_euc_d, qd_euc_d, ad_euc_d, qa_lev_d, qa_lev_r, qa_jar_s,
          qa_jaw_s, qa_tfidf_score, qd_tfidf_score, ad_tfidf_score, ques_tfidf_sum, txt_tfidf_sum, 
          ans_tfidf_sum, qa_jacc_simi, qd_jacc_simi, ad_jacc_simi]


In [0]:
def process_data(data=None,dataframe=False,to_csv=False,save=False,chunk_size=500):
  id_list=list()
  question_list=list()
  answer_list=list()
  target_list=list()
  features_list=list()
  for idx in tqdm(range(0,chunk_size)):
    id=df.loc[idx,'example_id']
    corpus=format_text(remove_html_tags(data.loc[idx,'text']))
    correct_text=format_text(remove_html_tags(data.loc[idx,'pred_long_ans']))
    question=format_text(remove_html_tags(data.loc[idx,'question']))
    possible_answers=format_txt(remove_html_tag(eval(data.loc[idx,'candidate_answers'])))
    

    tfidf=TfidfVectorizer(ngram_range=(1,2), stop_words=stop_words)
    tfidf.fit([corpus])
    text_tfidf=tfidf.transform([corpus]).todense()
    ques_tfidf=tfidf.transform([question]).todense()
    target=-1
    for poss_ans in possible_answers:
      ans_tfidf=tfidf.transform([poss_ans]).todense()
      feature=extract_features(text_tfidf,ques_tfidf,ans_tfidf,corpus,question,poss_ans)
    
      if poss_ans==correct_text:
        target=1
      else:
        target=0
      id_list.append(id)
      question_list.append(question)
      answer_list.append(correct_text)
      target_list.append(target)
      features_list.append(feature)
    
  return id_list,question_list,answer_list,target_list,features_list
      

In [0]:

id_list,question_list,answer_list,target_list,feature_list=process_data(data=df)

  9%|▉         | 46/500 [00:43<05:33,  1.36it/s]/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
 22%|██▏       | 110/500 [02:03<03:15,  1.99it/s]/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720

In [0]:
data=pd.DataFrame(columns=['example_id','target','question','answer','qa_cos_d', 'qd_cos_d', 'ad_cos_d', 'qa_euc_d', 'qd_euc_d', 'ad_euc_d', 
                           'qa_lev_d', 'qa_lev_r', 'qa_jar_s', 'qa_jaw_s', 'qa_tfidf_score', 'qd_tfidf_score', 'ad_tfidf_score',
                           'ques_tfidf_sum', 'txt_tfidf_sum', 'ans_tfidf_sum', 'qa_jacc_simi', 'qd_jacc_simi', 'ad_jacc_simi'])


In [0]:
for id,target,ques,ans,feature in tqdm(zip(id_list,target_list,question_list,answer_list,feature_list)):
  new_row={'example_id':id,'target':target,'question':ques,'answer':ans,'qa_cos_d':feature[0], 'qd_cos_d':feature[1], 'ad_cos_d':feature[2],
           'qa_euc_d':feature[3], 'qd_euc_d':feature[4], 'ad_euc_d':feature[5], 'qa_lev_d':feature[6], 'qa_lev_r':feature[7], 
           'qa_jar_s':feature[8], 'qa_jaw_s':feature[9], 'qa_tfidf_score':feature[10], 'qd_tfidf_score':feature[11], 'ad_tfidf_score':feature[12],
           'ques_tfidf_sum':feature[13], 'txt_tfidf_sum':feature[14], 'ans_tfidf_sum':feature[15], 'qa_jacc_simi':feature[16],
           'qd_jacc_simi':feature[17], 'ad_jacc_simi':feature[18]}
  data=data.append(new_row,ignore_index=True)

19601it [03:14, 100.61it/s]


In [0]:
data.to_csv('TF_QA_features_data.csv',header=True)


In [0]:
!cp TF_QA_features_data.csv "/content/drive/My Drive/TF_QA_DATA_SQuAD_FORMAT/"